In [1]:
import pandas as pd
import json
import pybedtools
import pathlib
import subprocess
import numpy as np
import xarray as xr

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/merge.py:18: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1829: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  'DataArray', pd.Series, pd.DataFrame, pd.Panel]:


In [2]:
#from dask.distributed import Client
#client = Client(dashboard_address=':5555')

In [3]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)
cell_tidy_data.index.name = 'cell'

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


## SubType Rate

In [4]:
mcds_dir = pathlib.Path('/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/')
mcds_list = list(mcds_dir.glob('*mcds'))

for path in mcds_list:
    mcds = xr.open_dataset(path).squeeze()
    use_cell = mcds.get_index('cell') & cell_tidy_data.index
    if use_cell.size == 0:
        continue
    print(path)
    mcds = mcds.sel(cell=use_cell)
    mcds.coords['CellClass'] = cell_tidy_data['CellClass']
    mcds.coords['MajorType'] = cell_tidy_data['MajorType']
    mcds.coords['SubType'] = cell_tidy_data['SubType']
    
    sub_type_sum = mcds.groupby('SubType').sum(dim='cell')
    sub_type_sum.to_netcdf(path.name + 'subtype_sum.nc')
    

/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10E-190625.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10E-190627.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10F-190625.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10F-190627.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11E-190214.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11E-190305.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11F-190214.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11F-190305.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/1A-180226.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/1A-180227.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gen

/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9H-190212.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9H-190219.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9J-190212.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9J-190219.geneslop2k.mcds


In [4]:
mcds_dir = pathlib.Path('./')
mcds_list = list(mcds_dir.glob('*subtype_sum.nc'))
unique_sub_types = pd.Index(cell_tidy_data['SubType'].unique(), name='SubType')

total_ds = xr.Dataset()
total_ds['geneslop2k_da'] = xr.DataArray([])
first = True
n = 0
for path in mcds_list:
    n += 1
    if n % 40 == 0:
        print(n)
    da = xr.open_dataset(path)['geneslop2k_da']
    if first:
        total_da = da.reindex({'SubType': unique_sub_types}).fillna(0)
        first = False
    else:
        da = da.reindex({k: total_da.coords[k] 
                         for k in ['SubType', 'geneslop2k', 'mc_type', 'count_type']}).fillna(0)
        total_da.values = total_da.values + da.values

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:267: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif isinstance(data, pd.Panel):


40
80


In [5]:
total_rate = total_da.sel(count_type='mc') / total_da.sel(count_type='cov')

ds = xr.Dataset({'gene_cluster_da': total_da, 
                 'gene_cluster_da_rate': total_rate})

ds.to_netcdf('SubType.geneslop2k.mcds')

In [6]:
assert int((ds['gene_cluster_da'].sel(count_type='mc') > ds['gene_cluster_da'].sel(count_type='cov')).sum()) == 0
assert int(np.isnan(ds['gene_cluster_da']).sum()) == 0

## MajorType Rate

In [7]:
mcds_dir = pathlib.Path(
    '/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/')
mcds_list = list(mcds_dir.glob('*mcds'))

for path in mcds_list:
    mcds = xr.open_dataset(path).squeeze()
    use_cell = mcds.get_index('cell') & cell_tidy_data.index
    if use_cell.size == 0:
        continue
    print(path)
    mcds = mcds.sel(cell=use_cell)
    mcds.coords['CellClass'] = cell_tidy_data['CellClass']
    mcds.coords['MajorType'] = cell_tidy_data['MajorType']
    mcds.coords['SubType'] = cell_tidy_data['SubType']

    cluster_sum = mcds.groupby('MajorType').sum(dim='cell')
    cluster_sum.to_netcdf(path.name + 'majortype_sum.nc')

/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10E-190625.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10E-190627.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10F-190625.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/10F-190627.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11E-190214.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11E-190305.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11F-190214.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/11F-190305.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/1A-180226.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/1A-180227.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gen

/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9H-190212.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9H-190219.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9J-190212.geneslop2k.mcds
/home/hanliu/project/mouse_rostral_brain/dataset/gene_with_2kb_slop/9J-190219.geneslop2k.mcds


In [8]:
mcds_dir = pathlib.Path('./')
mcds_list = list(mcds_dir.glob('*majortype_sum.nc'))
unique_sub_types = pd.Index(cell_tidy_data['MajorType'].unique(), name='MajorType')

total_ds = xr.Dataset()
total_ds['geneslop2k_da'] = xr.DataArray([])
first = True
n = 0
for path in mcds_list:
    n += 1
    if n % 40 == 0:
        print(n)
    da = xr.open_dataset(path)['geneslop2k_da']
    if first:
        total_da = da.reindex({'MajorType': unique_sub_types}).fillna(0)
        first = False
    else:
        da = da.reindex({k: total_da.coords[k] 
                         for k in ['MajorType', 'geneslop2k', 'mc_type', 'count_type']}).fillna(0)
        total_da.values = total_da.values + da.values
        
total_rate = total_da.sel(count_type='mc') / total_da.sel(count_type='cov')

ds = xr.Dataset({'gene_cluster_da': total_da, 
                 'gene_cluster_da_rate': total_rate})

ds.to_netcdf('MajorType.geneslop2k.mcds')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:267: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif isinstance(data, pd.Panel):


40
80


In [9]:
assert int((ds['gene_cluster_da'].sel(count_type='mc') > ds['gene_cluster_da'].sel(count_type='cov')).sum()) == 0
assert int(np.isnan(ds['gene_cluster_da']).sum()) == 0